In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Lobby as LobbyModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel
from app.models import LobbySeed as LobbySeedModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

Models = [
    BracketModel, 
    LobbyModel,
    TournamentModel, 
    MatchModel, 
    RoundModel, 
    UserModel, 
    LobbySeedModel
]

for Model in Models:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
# log miguel in
url = 'http://127.0.0.1:5000/api/login'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"miguel logged in"}\n'

In [10]:
# log miguel out
url = 'http://127.0.0.1:5000/api/logout'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"logged out"}\n'

In [11]:
url = 'http://127.0.0.1:5000/api/users'
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })
    r = requests.post(url, data=payload, headers=headers)


In [12]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [13]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4Mzc5NzczOCwiZXhwIjoxNTgzNzk4MzM4fQ.eyJpZCI6MX0.iUqlmgwLtyDWZ3hYzWSmMPo861CK0_1BuJGQ99AVXnHTSjX496NdYr-luZdygqU9OWhKRn6WuMqsekZkw9imQw'

In [14]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [15]:
# miguel creates lobby

url = 'http://127.0.0.1:5000/api/create/lobby/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "tournament_name" : "miguel's tourneys"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Lobby Created': "Lobby created for tournament: miguel's tourneys"}

In [47]:
q = \
"""
    select * from lobby_seeds where lobby_id == 1
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,user_id,lobby_id,seed
0,1,1,1,6
1,2,5,1,1
2,3,4,1,2
3,4,9,1,8
4,5,8,1,7
5,6,6,1,3
6,7,3,1,9
7,8,7,1,4
8,9,2,1,5


In [22]:
q = \
"""
    select * from lobby
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourneys,None,1


In [48]:
q = \
"""
    select * from user
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$eVYKoiUq$2fcd8220f906de3e...,None,2020-03-09 23:48:55.878780
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$QrfccSIt$4395d40c72452334...,None,2020-03-09 23:48:57.424065
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$HTsA9PNl$c53c4d815bc13668...,None,2020-03-09 23:48:57.510157
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$q2Wqu4pd$f05942ae472c0b02...,None,2020-03-09 23:48:57.594605
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$XT8tAoJi$07b8929a24747825...,None,2020-03-09 23:48:57.677475
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$MHZVyIJ1$7253d9c2a6706395...,None,2020-03-09 23:48:57.761094
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$OQpp5pme$96e41b0305e1ef4e...,None,2020-03-09 23:48:57.841813
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$PReLerqJ$cb978f4f23cc74c0...,None,2020-03-09 23:48:57.927374
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$bN1NUtVU$085e56604a557eb4...,None,2020-03-09 23:48:58.009338
9,10,Guest,guesty,None,None,None,2020-03-09 23:54:28.762029


In [28]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4Mzc5Nzc3OSwiZXhwIjoxNTgzNzk4Mzc5fQ.eyJpZCI6MX0.IgqYhp4gnJoxeFke-UI-sEj3I8zad_Tuijm5wzDJ9tNY3YZV-6djMBUbYutqcR3qgTNTxEvXcPdNTZCiCudRMQ'

In [29]:
users = df.username.values
np.random.shuffle(users)
import random

# miguel adds users to his lobby

url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

seeds = random.sample(range(1, len(users)+1), len(users)) 
roles = ["User" for u in users]
for u, s, r in zip(users, roles, seeds):
    print(f'adding {u}, {s}, {r}')
    payload = json.dumps({
        "username" : u,
        "role" : s,
        "seed" : r
    })
    r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
    print(json.loads(r.content))
    

adding Kevin, User, 6
{'Unique Constraint Failed': '<User Kevin> already in <Lobby 1>'}
adding Ptolemy, User, 4
{'Unique Constraint Failed': '<User Ptolemy> already in <Lobby 1>'}
adding miguel, User, 1
{'Unique Constraint Failed': '<User miguel> already in <Lobby 1>'}
adding Sunrisebanana, User, 9
{'Unique Constraint Failed': '<User Sunrisebanana> already in <Lobby 1>'}
adding Spaceghost, User, 3
{'Unique Constraint Failed': '<User Spaceghost> already in <Lobby 1>'}
adding AngryFalco, User, 8
{'Unique Constraint Failed': '<User AngryFalco> already in <Lobby 1>'}
adding Vik, User, 5
{'Unique Constraint Failed': '<User Vik> already in <Lobby 1>'}
adding TPN, User, 2
{'Unique Constraint Failed': '<User TPN> already in <Lobby 1>'}
adding Burnaby, User, 7
{'Unique Constraint Failed': '<User Burnaby> already in <Lobby 1>'}


In [53]:
url = 'http://127.0.0.1:5000/api/lobby/1'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
json.loads(r.content)

{'id': 1, 'tournament_name': "miguel's tourneys"}

In [55]:
# miguel adds a guest user

url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestb",
    "role" : "Guest",
    "seed" : 10
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
print(json.loads(r.content))

{'Added': 'Guest guestb to lobby 1'}


In [56]:
# log miguel out
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# TPN tries to add his own guests to miguel's lobby
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestx",
    "role" : "Guest",
    "seed" : 1
})
r = requests.post(url, data=payload, headers=headers, auth=('TPN', 'tpn'))
print(json.loads(r.content))

UserModel \
    .query \
    .filter_by(username='guesta', role='Guest') \
    .first() is None

{'Unauthorized': 'TPN cannot add entrants to this lobby'}


False

In [57]:
# log miguel out
url = 'http://127.0.0.1:5000/api/logout'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"logged out"}\n'

In [58]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('TPN', 'tpn'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4Mzc5ODI3MCwiZXhwIjoxNTgzNzk4ODcwfQ.eyJpZCI6Mn0.-tzeSgPOKJo5R_ewIBQ6D4wxWXplR7JiUPYduE65FcjV7EOz1c3urv9ss_ST0gWcImfCkQXKAicukYgXUhxGiA'

In [59]:
# TPN tries to add his own guests to miguel's lobby
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestaas",
    "role" : "Guest",
    "seed" : 1
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
print(json.loads(r.content))


{'Unauthorized': 'TPN cannot add entrants to this lobby'}


In [68]:
url = 'http://127.0.0.1:5000/api/lobby/1'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
content =  json.loads(r.content)
content

{'id': 1, 'tournament_name': "miguel's tourneys"}

In [67]:
q = \
"""
    select user_id, seed from lobby_seeds where lobby_id == 1
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,user_id,seed
0,1,6
1,5,1
2,4,2
3,9,8
4,8,7
5,6,3
6,3,9
7,7,4
8,2,5
9,12,10


In [166]:
url = 'http://127.0.0.1:5000/api/lobby/1/entrants'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
content =  json.loads(r.content)
content

[{'user': 1, 'lobby': 1, 'seed': 6, 'id': 1},
 {'user': 5, 'lobby': 1, 'seed': 1, 'id': 2},
 {'user': 4, 'lobby': 1, 'seed': 2, 'id': 3},
 {'user': 9, 'lobby': 1, 'seed': 8, 'id': 4},
 {'user': 8, 'lobby': 1, 'seed': 7, 'id': 5},
 {'user': 6, 'lobby': 1, 'seed': 3, 'id': 6},
 {'user': 3, 'lobby': 1, 'seed': 9, 'id': 7},
 {'user': 7, 'lobby': 1, 'seed': 4, 'id': 8},
 {'user': 2, 'lobby': 1, 'seed': 5, 'id': 9},
 {'user': 12, 'lobby': 1, 'seed': 10, 'id': 10}]

In [115]:
LobbySeedModel.query.filter_by(lobby_id=1).all()

[<LobbySeed 1>,
 <LobbySeed 2>,
 <LobbySeed 3>,
 <LobbySeed 4>,
 <LobbySeed 5>,
 <LobbySeed 6>,
 <LobbySeed 7>,
 <LobbySeed 8>,
 <LobbySeed 9>,
 <LobbySeed 10>]